In [2]:

%pip install pandas
%pip install geopy
%pip install numpy
%pip install gurobipy




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd


def load_total_demand():
  yearly_sales = pd.read_csv('main_data/vw_yearly_sales.csv')
  return yearly_sales


def load_dcs():
  # 2 samples
  # dcs= pd.read_csv('small_list_of_dcs.csv', encoding= 'unicode_escape', names=['ID', "Name","City","State","Address Line 1","Address Line 2", "Coordinates", "Area m3", "Facility Type", "StateConst"])

  # FULL DATASET
  dcs = pd.read_csv('main_data/list_of_dcs.csv', encoding= 'unicode_escape', names=['ID', "Name","City","State","Address Line 1","Address Line 2", "Coordinates", "Area m3", "Facility Type", "StateConst"])
  #stateconst is a constant used to account for the different prices for the cost of living and rent in various states

  dcs['Area sqft'] = round(dcs['Area m3'] * 10.7639)
  return dcs



def load_dealers():
  # https://www.vw.com/app/dccsearch/vw-us/en/Find%20a%20Volkswagen%20Dealer/+/38.353354499999995/-95.3817145/3/+/+/+/+

  # 2 SAMPLES
  # dealers= pd.read_csv('small_list_of_dealers.csv',names=['ID', "Name","Address Line 1","Address Line 2", "Website","Phone Number", "Coordinates"])

  # FULL DATASET
  dealers= pd.read_csv('main_data/list_of_dealers.csv',names=['ID', "Name","Address Line 1","Address Line 2", "Website","Phone Number", "Coordinates"])
  return dealers


def load_plants():
  plants= pd.read_csv('main_data/list_of_vw_plants_and_products.csv', encoding= 'unicode_escape',names=['ID', "City","Country", "Model","Coordinates","Current/Former plant"])

  plants_usa = plants.loc[plants["Country"]=="USA"]

  return plants_usa

def load_dealer_demands():

  def single_sample():

  # 4 samples
    dealers= pd.read_csv('Preliminary_data/small_list_of_demand_data.csv',names=['ID',"Name","Rating","Proportion","na"])
    del dealers[dealers.columns[-1]]
    return dealers

  def full_data():
  # FULL DATASET
    dealers= pd.read_csv('main_data/list_of_demand_data.csv',names=['ID',"Name","Rating","Proportion","na", "na2", "na3"])
    del dealers[dealers.columns[-1]]
    del dealers[dealers.columns[-1]]
    del dealers[dealers.columns[-1]]
    return dealers

  dealers = full_data() # either call single_sample or full_data

  return dealers


In [4]:

load_dealer_demands() # used google ratings as a indicator of demand/traffic


FileNotFoundError: [Errno 2] No such file or directory: 'data/list_of_demand_data.csv'

In [ ]:
load_total_demand()

In [ ]:
load_dcs()

In [ ]:
load_dealers()

In [ ]:
load_plants()

In [ ]:
### do location work
from geopy.distance import geodesic
import numpy

# ALL DISTANCES ARE IN MILES!!!!
# ALL AREAS ARE IN SQFT!!!
def dc_to_dealer_distances(dcs, dealers): # using geodesic distance, may want to add 15% or something onto distance
  distance_dc_to_dealer = numpy.zeros((len(dcs), len(dealers)))
  for dc_id, dc in dcs.iterrows():
    for dealer_id, dealer in dealers.iterrows():
      point1 = dc["Coordinates"].split(",")
      point2 = dealer["Coordinates"].split(",")
      distance = geodesic(point1, point2).miles
      dc_id = dc["ID"]
      dealer_id = dealer["ID"]
      # print ("distance from dc " + str(dc_id)+ " to dealer "+ str(dealer_id) + " is " + str(distance))
      distance_dc_to_dealer[dc_id][dealer_id] = distance
  distance_dc_to_dealer_df = pd.DataFrame(distance_dc_to_dealer)
  return distance_dc_to_dealer_df


def plant_to_dc_distances(plants, dcs):
  # get coordinates of the 1 usa plant
  plant_coordinates = plants['Coordinates'].iloc[0].split(",")
  distance_plants_to_dcs = [] # since we only have 1 usa plant we can just make everything from there (very convenient)
  for dcs_id, dc in dcs.iterrows():
    point1 = plant_coordinates
    point2 = dc["Coordinates"].split(",")
    distance = geodesic(point1, point2).miles
    dc_id = dc["ID"]
    distance_plants_to_dcs.append(distance)
  return distance_plants_to_dcs

plants = load_plants()
dcs = load_dcs()
dealers = load_dealers()

dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
plants_to_dcs = plant_to_dc_distances(plants, dcs)


print(plants_to_dcs)
print(dcs_to_dealers)

In [ ]:
plant_to_dc_distances(plants, dcs)

In [ ]:
## Calculate demands
def get_dealer_demands(year):
  dealers = load_dealers()
  dealer_demands = load_dealer_demands()
  total_demands = load_total_demand() # demand for each specific car type


  def get_demands(year):
    year_demands = total_demands.loc[total_demands['year'] == year]
    year_demands = year_demands[["Atlas","Cross Sport","ID4","Passat"]]
    return year_demands

  year_demands = get_demands(2021)
  year_demands = year_demands.values.tolist()[0]

  # print(year_demands)

  proportion_sum = dealer_demands["Proportion"].sum()

  dealer_demands [["Ratio of sales"]] = None
  dealer_demands["Ratio of sales"] = dealer_demands["Proportion"]/proportion_sum

  # print(dealer_demands)

  # # with the sales proportion, get the general sales for each location

  temp_ratio_col = pd.DataFrame(dealer_demands["Ratio of sales"])

  temp = []
  for x in list(dealer_demands["Ratio of sales"]):
    temp1=[]
    for y in list(year_demands):
      temp1.append(round(x*y))
    temp.append(temp1)
  sales = pd.DataFrame(temp, columns=["Atlas","Cross Sport","ID4","Passat"])
  return sales

In [ ]:
sales = get_dealer_demands(2021)
sales
# the left side are the dealer ids

In [ ]:
## Insert something here with defining facilities


def calculate_dc_cost():


  def calculate_dc_cost_monthly(id, state, facility_type, areasqft):
  # print (id, facility, state, areasqft)
    cost = 1.5*areasqft #2$ baseline
    insurance = 3850  # https://glcdistribution.com/customer-resources/tools/warehousing-insurance-calculator/ 1 million dollar standard

    employee_salaries = (areasqft/2000)*20*(40*4) # https://www.payscale.com/research/CA/Job=Warehouse_Material_Handler/Hourly_Rate

  # pay 1 dollar per sqft for utility
    utilities = areasqft


  # $600 – $950 per month to lease a $50,000 forklift.
  # https://discord.com/channels/260272353118912522/1093655130076893284/1129635416409133076 1 forklift services 5000 sqft
    forklift_cost =  areasqft/5000  *800


    # https://discord.com/channels/260272353118912522/1093655130076893284/1129639090942398504
    price_it_costs_to_build_warehouse_sqft = 100*areasqft
    property_tax = 6/1000*price_it_costs_to_build_warehouse_sqft


    security = 1000 + (5 * 8 * 28*20) # 1000 fixed cost monitoring + 20$/h * 40h/week * 28 people

    if facility_type == 1:
      ft = 1
    else:
      ft = 1.2

    price = (cost + insurance + employee_salaries + utilities + forklift_cost + security + property_tax)* (state/64187)*ft
    # 508 is the price of 100 big macs and state is the price of 100 big macs in that state
    return price


  prices = []

  for i in range(len(dcs.index)):
      moneys = calculate_dc_cost_monthly(float(dcs.loc[i,["ID"]]), float(dcs.loc[i,["StateConst"]]), float(dcs.loc[i,["Facility Type"]]),float(dcs.loc[i,["Area sqft"]] ))
      prices.append(moneys*12)
  return prices
  # print(calculate_dc_cost())


# C_i
def calculate_dc_shipment_capacity():
  dcs = load_dcs()
  dcs = dcs[["Area sqft",'Facility Type']]


# The largest car to consider is the Cross Sport
# It has dimensions: 4,966 mm L x 1,990 mm W x 1,723 mm H
# we can round this up to: 17 ft length, 7 foot wide, 6 foot tall
# add about 2 ft to everything as a buffer
# new measurement is about 20 ft length, 10ft wide, 8 foot tall
# so, we can predict that 2 car takes up 200/2, each car takes 100 sq foot

# from https://discord.com/channels/260272353118912522/1093655130076893284/1131271495004459018,
# we see that 60-80% of DCs are for storage. Let's assume 60%


  vehicle_capacity = dcs["Area sqft"]*0.5 /100


# from https://discord.com/channels/260272353118912522/1093655130076893284/1131266537869820075
# we see that most vehicles stay in the dc a couple of weeks (take 2 weeks)
# we can frame this as: All vehicles in the DCS arrive at time 0, and after 2 weeks all are released

  annual_throughput = round(vehicle_capacity) * (52/2)

  # dcs["Annual Throughput"] = annual_throughput

  # print(annual_throughput)
  return (list(annual_throughput))

# WE NEED TO BE CAREFUL ABOUT TYPE 2 FACILITIES:

# print(calculate_dc_cost())

# print(calculate_dc_shipment_capacity())

# Important Verbiage

Has most of the answers:

def generate_facility_cost

---
### Generally 2-3$ per square foot
### https://www.warehousingandfulfillment.com/resources/fulfillment-cost-calculator/

1. Rent or Mortgage:
[A 50,000 square foot steel warehouse, in the words of one construction expert, will cost between 52,800 and 312,500 per month in addition to monthly rent, utilities, insurance, and other associated costs. A 100,000 increase in the monthly budget will be made up of the cost of a turnkey warehouse, which will also be included in the estimate.](https://www.hechtgroup.com/the-average-size-of-a-small-warehouse-is-1-000-5-000-square-feet/)


2. Insurance:
- a standard warehouse general liability is 1-2 million of coverage, which is about 100/month
3. Salaries:
```
There is no definitive answer to how many employees per square foot warehouse as it depends on a variety of factors such as the type of products being stored, the level of automation in the warehouse, and the workflows. However, a good rule of thumb is to have one employee for every 1,000-1,500 square feet of warehouse space. This allows for efficient operations and ensures that there is enough staff to handle the volume of work.Every employee in a warehouse is allocated about 200-250 square feet of space. Distribution centers had a median square footage of 240,410 in 2016, 176,600 in 2017, and 305,000 in 2018. If a warehouse is properly designed, it will be more efficient and require less space per employee than if it is poorly designed. The number of warehouse employees varies greatly. Warehouses in general are less than 10,000 square feet in size. Eight-hour shifts pay between $14.80 and $19.90 per hour on average. Approximately 35% of the total warehouse space is estimated to be occupied by this type of warehouse.
```
`https://www.hechtgroup.com/how-many-employees-should-a-warehouse-have-per-square-foot/`

4. Utilities:
- [average utlity cost per square foot commercial property is about 1 dollar/sqft for electricity](https://www.iotacommunications.com/blog/average-utility-cost-per-square-foot-commercial-property/)
5. Equipment:
875$ lease a forklift

6. Licensing and Permits:

7. Maintenance and Repairs:

8. Property Taxes:

9. Security:

10. Technology:




# Back to Code

In [ ]:
## Determining the cost of transportation


## THE FUNCTION OF SHIPPING TRUCKS CAN BE APPROXIMATED BY A PIECEWISE FUNCTION
# https://www.desmos.com/calculator/ipxbog2uyr
# constructed from the data here: https://www.marketwatch.com/guides/car-shipping/truck-shipping/#:~:text=According%20to%20quotes%20we%20received,when%20using%20an%20enclosed%20carrier.

def fixed_cost_shipment(): #T_1
  return 186 # cost to drive 0 miles

def calculate_variable_cost_shipment(mileage): #T_2
  # if mileage < 1135:
  #   return mileage * (551/535)
  # return mileage * (0.43)
  return (0.43)

  # TODO: WE ARE ASSUMING FOR SIMPLICYT SAKE 0.43$/MILE AND ARE NOT CONSIDERING THE OTHER FUNCTION FOR MODELLING RIGHT NOW






In [ ]:
## The modelling?
import gurobipy as gp
from gurobipy import GRB


# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '47fa1c45-5377-4f93-87d4-3da7da2b6955')
e.setParam('WLSSECRET', '7dfbddcb-7333-4ec7-927d-b459cffd9565')
e.setParam('LICENSEID', 868415)
e.start()


##LOAD DATA

# plants = load_plants()
# dcs = load_dcs()
# dealers = load_dealers()

# dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
# plants_to_dcs = plant_to_dc_distances(plants, dcs)
# dealer_demands = get_dealer_demands(2021)
# dc_annual_cost = calculate_dc_cost()





plants = load_plants()
dcs = load_dcs()
dealers = load_dealers()

dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
plants_to_dcs = plant_to_dc_distances(plants, dcs)
dealer_demands = get_dealer_demands(2021) ## car demand for each dealership
dc_annual_cost = calculate_dc_cost()



# define sets
plants_set = [0] # set S
dcs_set = list(dcs["ID"]) # set I
dealers_set = list(dealers["ID"]) # set J
vehicle_set = [0,1,2,3] # we define 0 to 3, corresponding to Atlas, CrossSport, ID4 and Passat vehicles (set V)
facility_type_set = [0, 1] # set P

In [ ]:
dcs_set

In [ ]:
# Parameter prep - we change the dataframes to arrays of sorts, which we can use indexes and such in the formulation

d_jv = dealer_demands.values
# d_jv[1][0]

m_ij = dcs_to_dealers.values

# dc 1 to dealer 2 should be 1475.6
# m_ij[1][2]

c_si = plants_to_dcs

L_j = 1 # load factor is 1

T_1 = fixed_cost_shipment()

T_2 = calculate_variable_cost_shipment("dummy_value") # see code comments on why we do this as a temp hack for a preliminary model

f_ip = dc_annual_cost

C_i = calculate_dc_shipment_capacity()

In [ ]:
d_jv

In [ ]:
m_ij

In [ ]:
c_si

In [ ]:
L_j

In [ ]:
T_1

In [ ]:
T_2

In [ ]:
f_ip

In [ ]:
C_i

In [ ]:
m = gp.Model(env=e)

# CREATE THE DECISION VARIABLES HERE
# : Annual shipment of type v∈V vehicles from source s∈S to DC location i∈I
x1_siv = m.addVars(plants_set, dcs_set, vehicle_set, vtype=GRB.INTEGER) #
x2_ijv = m.addVars(dcs_set, dealers_set, vehicle_set, vtype=GRB.INTEGER )
y_ip = m.addVars(dcs_set, facility_type_set, vtype=GRB.BINARY)

In [ ]:
x1_siv

In [ ]:
x2_ijv

In [ ]:
y_ip

In [ ]:
# TODO: FIX THIS
#  m.setObjective(, gp.GRB.MINIMIZE)

# obj2_sub1 = sum(x2_ijv[i, j, v] + x1_siv[s, i, v] for v in vehicle_set for j in dealers_set for s in plants_set for i in dcs_set)
# obj2 = sum(obj2_sub1 * c_si[i] for s in plants_set for i in dcs_set)

# Need to verify them all with Affleck
obj1 = sum((x2_ijv[i, j, v]/L_j)*(T_1 + T_1*m_ij[i, j]) for v in vehicle_set for j in dealers_set for i in dcs_set)

# Since we only have one plant, c_si should only take in i as an index. We are good.
obj2 = sum(sum(x2_ijv[i, j, v] + x1_siv[s, i, v] for v in vehicle_set for j in dealers_set for s in plants_set) * c_si[i] for i in dcs_set)

obj3 = sum(x1_siv[s, i, v] for v in vehicle_set for s in plants_set for i in dcs_set)

# This works for a small example, but not for a larger example
# Can we just do one type of facility?
obj4 = sum(f_ip[i]*y_ip[i, p] for p in facility_type_set for i in dcs_set)

m.setObjective(obj1 + obj2 + obj3 + obj4, gp.GRB.MINIMIZE)
#m.update()
#m.display()

In [ ]:
# Constraints:

# Constraint 1: Demand at each market area for each vehicle type must be met
m.addConstrs(sum(x2_ijv[i, j, v] for i in dcs_set) == d_jv[j, v] for v in vehicle_set for j in dealers_set)

# Constraint 2: Vehicle flows between plants and DCs must be conserved
# Ask Affleck about summing over i twice

m.addConstrs(sum(x1_siv[s, i, v] for s in plants_set) == sum(x2_ijv[i, j, v] for j in dealers_set) for i in dcs_set for v in vehicle_set)

# Constraint 3: Total vehicle flow to each DC must satisfy the minimum capacity requirement
m.addConstrs(
    (C_i[i] * y_ip[i, 1] <= sum(x2_ijv[i, j, v] for v in vehicle_set for j in dealers_set) for i in dcs_set),
    name="constraint_3_min"
)

# Constraint 3: Total vehicle flow to each DC must satisfy the maximum capacity requirement
m.addConstrs(
    (sum(x2_ijv[i, j, v] for v in vehicle_set for j in dealers_set) <= C_i[i] * y_ip[i, 0] + sum(d_jv[j, v] for v in vehicle_set for j in dealers_set) * y_ip[i, 1] for i in dcs_set),
    name="constraint_3_max"
)

# # Constraint 4: Shipment quantities must be nonnegative
m.addConstrs(x1_siv[s, i, v] >= 0 for s in plants_set for v in vehicle_set for i in dcs_set)
m.addConstrs(x2_ijv[i, j, v] >= 0 for i in dcs_set for v in vehicle_set for j in dealers_set)

# Constraint 6: Ask Affleck
r=60

m.addConstrs((sum(x2_ijv[i, j, v] for v in vehicle_set) == 0 for i in dcs_set for j in dealers_set if m_ij[i][j] / 300 > r)
)


In [ ]:
m.optimize()
# m.write("out.sol")

In [ ]:
# m.display()

In [ ]:

# PRINT VALUES OF THE DV HERE:


x1_siv


In [ ]:
x2_ijv

In [ ]:
y_ip

In [ ]:
# Also, if there is no DC installed at a location, there is no shipment to that location

In [ ]:
# The distribution centers must be selected so that all market areas can be reached within r days. Suppose a truck travels on average 300 miles a day.